# Настройка среды и проверка версий

In [1]:
!python -V

Python 3.9.21


In [2]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:42:46_Pacific_Standard_Time_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.current_device()

0

In [6]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1650'

In [7]:
torch.backends.cudnn.version()

90701

In [8]:
torch.backends.cudnn.enabled

True

In [9]:
print(torch.version.cuda)

12.8


In [10]:
import torchvision
print(torchvision.__version__)
print(torchvision.__file__)

0.22.0.dev20250412+cu128
D:\anaconda\envs\yolo_det\lib\site-packages\torchvision\__init__.py


# Превращение лейблов и структуры папок в COCO format под YOLO формат

In [ ]:
import json
import os
from tqdm import tqdm

with open("../Desktop/dataset_fire_smoke/475_fire_train/annotations/instances_default.json") as f:
    coco = json.load(f)

categories = {cat["id"]: cat["name"] for cat in coco["categories"]}

images = {img["id"]: img for img in coco["images"]}

image_to_annotations = {}
for ann in coco["annotations"]:
    img_id = ann["image_id"]
    image_to_annotations.setdefault(img_id, []).append(ann)

class_names = list(categories.values())
class_to_id = {name: i for i, name in enumerate(class_names)}

# Путь для данных в yolo формате
os.makedirs("../Desktop/dataset_fire_smoke_YOLO/labels/train", exist_ok=True)

# Итерация и сохранение лейблов
for img_id, img_info in tqdm(images.items()):
    file_name = os.path.basename(img_info["file_name"]).replace(".jpg", ".txt")
    label_path = os.path.join("../Desktop/dataset_fire_smoke_YOLO/labels/train", file_name)
    img_w, img_h = img_info["width"], img_info["height"]

    anns = image_to_annotations.get(img_id, [])
    yolo_lines = []

    for ann in anns:
        cat_id = ann["category_id"]
        bbox = ann["bbox"]
        class_name = categories[cat_id]
        class_id = class_to_id[class_name]

        x, y, w, h = bbox
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w /= img_w
        h /= img_h

        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    with open(label_path, "w") as f:
        f.write("\n".join(yolo_lines))


100%|██████████| 1142/1142 [00:01<00:00, 1080.42it/s]


In [ ]:
import json
import os
from tqdm import tqdm

with open("../Desktop/dataset_fire_smoke/474_fire_val/annotations/instances_default.json") as f:
    coco = json.load(f)

categories = {cat["id"]: cat["name"] for cat in coco["categories"]}
images = {img["id"]: img for img in coco["images"]}

image_to_annotations = {}
for ann in coco["annotations"]:
    img_id = ann["image_id"]
    image_to_annotations.setdefault(img_id, []).append(ann)

class_names = list(categories.values())
class_to_id = {name: i for i, name in enumerate(class_names)}

# Путь для данных в yolo формате
os.makedirs("../Desktop/dataset_fire_smoke_YOLO/labels/val", exist_ok=True)

# Итерация и сохранение лейблов
for img_id, img_info in tqdm(images.items()):
    file_name = os.path.basename(img_info["file_name"]).replace(".jpg", ".txt")
    label_path = os.path.join("../Desktop/dataset_fire_smoke_YOLO/labels/val", file_name)
    img_w, img_h = img_info["width"], img_info["height"]

    anns = image_to_annotations.get(img_id, [])
    yolo_lines = []

    for ann in anns:
        cat_id = ann["category_id"]
        bbox = ann["bbox"]  # [x, y, width, height]
        class_name = categories[cat_id]
        class_id = class_to_id[class_name]

        x, y, w, h = bbox
        # Convert to YOLO format: normalized center x/y and width/height
        x_center = (x + w / 2) / img_w
        y_center = (y + h / 2) / img_h
        w /= img_w
        h /= img_h

        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}")

    with open(label_path, "w") as f:
        f.write("\n".join(yolo_lines))


100%|██████████| 308/308 [00:00<00:00, 1350.90it/s]


In [ ]:
import yaml

data = {
    "train": "../Desktop/dataset_fire_smoke_YOLO/images/train",  # update with your actual path
    "val": "../Desktop/dataset_fire_smoke_YOLO/images/val",      # update with your actual path
    "nc": 7,
    "names": [
        "motorcycle",
        "bicycle",
        "wheelchair",
        "smoke",
        "fire",
        "scooter",
        "person"
    ]
}

with open("../Desktop/dataset_fire_smoke_YOLO/data.yaml", "w") as f:
    yaml.dump(data, f, default_flow_style=False)

print("✅ data.yaml created!")

✅ data.yaml created!


# Настройка YOLO

In [19]:
!pip install ultralytics

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/974.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/974.5 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 974.5/974.5 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ----------------- ---------------------- 3.4/7.8 MB 22.3 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 19.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   -- ------------------------------------- 2.9/39.5 MB 42.3 MB/s eta 0:00:01
   ------- -------------------------------- 7.1/39.5 MB 19.0 MB/s eta 0:00:02
   ---------- ----------------------------- 10.0/39.5 MB 17.7 MB/s eta 0:00:02
   ----------- ---------------------------- 11.8/39.5 MB 15.1 MB/s eta 0:00:02
   -------------- ------------------------- 14.7/39.5 MB 14.9 MB/s eta 0:00:02
   ----------------

In [11]:
import ultralytics

In [12]:
from ultralytics import YOLO

In [21]:
model = YOLO("yolo11s.pt")

100%|██████████| 18.4M/18.4M [00:04<00:00, 4.19MB/s]


In [26]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [27]:
train_results = model.train(
    data="C://Users/Temirlan/Desktop/dataset_fire_smoke_YOLO/data.yaml",  # Path to dataset configuration file
    epochs=55,  # Number of training epochs
    batch=2,
    imgsz=640,  # Image size for training
    device=0,  # Device to run on (e.g., 'cpu', 0, [0,1,2,3])
)

Ultralytics 8.3.107  Python-3.9.21 torch-2.8.0.dev20250411+cu128 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=C://Users/Temirlan/Desktop/dataset_fire_smoke_YOLO/data.yaml, epochs=55, time=None, patience=100, batch=2, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_l

train: Scanning C:\Users\Temirlan\Desktop\dataset_fire_smoke_YOLO\labels\train.cache... 1142 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1142/1142 [00:00<?, ?it/s]
val: Scanning C:\Users\Temirlan\Desktop\dataset_fire_smoke_YOLO\labels\val.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]


Plotting labels to runs\detect\train18\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train18
Starting training for 55 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/55      1.37G      1.547       1.66      1.485         15        640: 100%|██████████| 571/571 [01:57<00:00,  4.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:10<00:00,  7.19it/s]


                   all        308       2002      0.704      0.226      0.233      0.118

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      55/55       2.5G      1.076     0.7921      1.155          9        640: 100%|██████████| 571/571 [01:50<00:00,  5.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:07<00:00,  9.66it/s]

                   all        308       2002      0.743      0.289      0.357      0.195



55 epochs completed in 1.828 hours.
Optimizer stripped from runs\detect\train18\weights\last.pt, 19.2MB
Optimizer stripped from runs\detect\train18\weights\best.pt, 19.2MB

Validating runs\detect\train18\weights\best.pt...
Ultralytics 8.3.107  Python-3.9.21 torch-2.8.0.dev20250411+cu128 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11s summary (fused): 100 layers, 9,415,509 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:07<00:00, 10.15it/s]


                   all        308       2002      0.741      0.295      0.459      0.207
            motorcycle          5          8      0.665       0.25      0.343        0.2
               bicycle          2          2          1          0      0.496     0.0496
                 smoke        250        524      0.684      0.286      0.362      0.176
                  fire        303        761      0.623      0.511      0.556      0.288
                person        119        707      0.731      0.427       0.54       0.32
Speed: 0.8ms preprocess, 13.9ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train18
